### Importing Essential Libraries

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import time as tm

sb.set() # sets default Seaborn style for graphs

### Importing and Exploring the HDB Resale Flat Dataset

#### Reading the Dataset

In [2]:
dataobj = pd.read_csv("resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")
housingDF = pd.DataFrame(dataobj)

#### Exploring the Dataset

We first take a brief glance of how each entry in this dataset is like:

In [3]:
housingDF.head(5)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [4]:
housingDF.tail(5)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
70098,2020-03,YISHUN,EXECUTIVE,364,YISHUN RING RD,01 TO 03,146.0,Maisonette,1988,67 years 01 month,540000.0
70099,2020-03,YISHUN,EXECUTIVE,359,YISHUN RING RD,10 TO 12,146.0,Maisonette,1988,67 years 04 months,578888.0
70100,2020-03,YISHUN,EXECUTIVE,292,YISHUN ST 22,01 TO 03,165.0,Apartment,1992,71 years 03 months,660000.0
70101,2020-03,YISHUN,EXECUTIVE,611,YISHUN ST 61,04 TO 06,146.0,Maisonette,1987,66 years 08 months,620000.0
70102,2020-03,YISHUN,EXECUTIVE,827,YISHUN ST 81,01 TO 03,145.0,Maisonette,1987,66 years 07 months,660000.0


Next, we can check the data types of each column. We should be mindful about some columns such as "Block", which could either have an integer data type or an object data type (in the form of a string). 

In [5]:
housingDF.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
dtype: object

In [6]:
housingDF_year = housingDF.copy()

housingDF_year.rename(columns={'month' : 'year'}, inplace=True)
housingDF_year['year'] = housingDF_year['year'].str.slice(stop=4)

housingDF_year['year'].value_counts()

years = ['2017', '2018', '2019', '2020']

for year in years:
    housing_temp = housingDF_year[housingDF_year['year'] == year]
    print(housing_temp['resale_price'].describe(), "\n")

count    2.050900e+04
mean     4.438885e+05
std      1.491483e+05
min      1.750000e+05
25%      3.380000e+05
50%      4.100000e+05
75%      5.100000e+05
max      1.180000e+06
Name: resale_price, dtype: float64 

count    2.187500e+04
mean     4.406178e+05
std      1.570186e+05
min      1.600000e+05
25%      3.280000e+05
50%      4.070000e+05
75%      5.150000e+05
max      1.185000e+06
Name: resale_price, dtype: float64 

count    2.218800e+04
mean     4.321241e+05
std      1.539773e+05
min      1.500000e+05
25%      3.200000e+05
50%      4.000000e+05
75%      5.100000e+05
max      1.205000e+06
Name: resale_price, dtype: float64 

count    5.531000e+03
mean     4.344511e+05
std      1.501389e+05
min      1.400000e+05
25%      3.260000e+05
50%      4.080000e+05
75%      5.150000e+05
max      1.232000e+06
Name: resale_price, dtype: float64 



There doesn't seem to be much variation in the HDB resale flat prices from year to year. However, in order to ensure that the year isn't a confounder, we should just consider the most recent year, 2019:

In [7]:
housing_2019 = housingDF_year[housingDF_year['year'] == '2019']
housing_2019.head(5)

,year,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
42384,2019,ANG MO KIO,3 ROOM,330,ANG MO KIO AVE 1,01 TO 03,68.0,New Generation,1981,61 years 01 month,270000.0
42385,2019,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,04 TO 06,73.0,New Generation,1976,56 years 04 months,295000.0
42386,2019,ANG MO KIO,3 ROOM,225,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1978,58 years 01 month,270000.0
42387,2019,ANG MO KIO,3 ROOM,225,ANG MO KIO AVE 1,01 TO 03,67.0,New Generation,1978,58 years,230000.0
42388,2019,ANG MO KIO,3 ROOM,333,ANG MO KIO AVE 1,01 TO 03,68.0,New Generation,1981,61 years,262500.0


### Data Cleaning

Let's check if there are any form of redundancies by repeating the `street_name` when the `town` is already mentioned.

For instance, if every pair of `town` and `street_name` values are such that the `street_name` corresponding to the `town` are very similar (like the below example), we can consider dropping the `town` variable. 

| Town | Street Name | 
| ---- | ----------  |
| Yishun | Yishun Ring Rd |
| Yishun | Yishun St 22 | 
| Ang Mo Kio | Ang Mo Kio Ave 5 | 
| Ang Mo Kio | Ang Mo Kio Ave 10 | 

In [8]:
print(housing_2019['town'].value_counts(), "\n")
print(housing_2019['street_name'].value_counts())

SENGKANG           1795
WOODLANDS          1794
YISHUN             1790
JURONG WEST        1705
TAMPINES           1413
PUNGGOL            1160
BEDOK              1147
HOUGANG             967
ANG MO KIO          954
BUKIT PANJANG       937
CHOA CHU KANG       923
BUKIT MERAH         880
BUKIT BATOK         874
TOA PAYOH           706
SEMBAWANG           643
KALLANG/WHAMPOA     600
PASIR RIS           585
QUEENSTOWN          563
GEYLANG             536
JURONG EAST         500
CLEMENTI            479
SERANGOON           437
BISHAN              417
CENTRAL AREA        187
MARINE PARADE       125
BUKIT TIMAH          71
Name: town, dtype: int64 

YISHUN RING RD        336
YISHUN AVE 11         316
SEGAR RD              290
BEDOK RESERVOIR RD    231
ANG MO KIO AVE 3      222
                     ... 
ANG MO KIO ST 11        1
JLN BERSEH              1
KIM PONG RD             1
PASIR RIS ST 41         1
BT MERAH LANE 1         1
Name: street_name, Length: 535, dtype: int64


From the above results, there are some `street_names` that are less obvious in pointing out their corresponding `town`. Therefore, we should not drop the `town` column from the dataset. 

Let's check if there are any missing values in the dataset: 

In [9]:
housing_2019.isnull().sum()

year                   0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
dtype: int64

We observe that many of the columns in this dataset are categorical. Some models (such as linear models) require numerical inputs, therefore there is the need to create new columns such that the values are substituted with numerical ones.

In [10]:
print("Total one hot encoded columns :") # check how many more columns have to be created for each column via one-hot encoding
for x in housing_2019.columns:
    print(housing_2019[x].nunique())
print("Size : ", housing_2019.shape)

Total one hot encoded columns :
1
26
7
2252
535
17
147
19
51
601
1443
Size :  (22188, 11)


This seems like an awful lot of columns to one-hot encode. We can definitely do further data cleaning in order to allow both the results to arrive faster and to also get rid of noisy data which might potentially mess up our predictions.

One particular column which we might not need for the models we are implementing on our case would be the `block` column. This is because we are not going to get involved with geographic data in this case, which is not applicable in some models such as linear models, so I would have a spare dataset without the `block` column.

In addition, since we have the `month` column (which actually displays the year and month of sale price), as well as the `lease_commence_date` (lease commencement date) and the `remaining_lease` are highly correlated to one another (since subtracting the `lease_commence_date` from `month` would give us the `remaining_lease` column, to simplify things, we could consider dropping the former two columns as well.

### Implementing the models

#### Random Forest

In [11]:
# allvalues = dataobj.values
# housingdata = allvalues[:, :]

# housingdata_noblock = np.delete(housingdata.copy(), 3, axis=1)
# housingdata_mod = np.delete(housingdata.copy(), [0, 3, 9], axis=1)

# housingtarget = allvalues[:, -1].astype("float")

# encoderobj = OneHotEncoder(sparse = False)
# housingdata_mod = encoderobj.fit_transform(housingdata_mod[:15000]).astype("float")
# housingtarget = housingtarget[:15000]
# xtrain, xtest, ytrain, ytest = train_test_split(housingdata_mod, housingtarget, random_state = 8)
# rfrobj = RandomForestRegressor(n_estimators = 10, random_state = 8)
# rfrobj.fit(xtrain, ytrain)


housing_dummies = pd.get_dummies(housing_2019.loc[:,'year':'remaining_lease'], drop_first=True) # drops the original column after one-hot encoding is done
housing_target = housing_2019['resale_price']
xtrain, xtest, ytrain, ytest = train_test_split(housing_dummies, housing_target, random_state = 8)

encoderobj = OneHotEncoder(sparse = False)
housingdata_mod = encoderobj.fit_transform(housing_dummies).astype("float")
rfrobj = RandomForestRegressor(n_estimators = 10, random_state = 8)
rfrobj.fit(xtrain, ytrain)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=8, verbose=0, warm_start=False)

In [17]:
print("For Random Forest: ")
print("Training set accuracy: ")
print(rfrobj.score(xtrain, ytrain))
print("Test set accuracy: ")
print(rfrobj.score(xtest, ytest))

For Random Forest: 
Training set accuracy: 
0.9905292335776913
Test set accuracy: 
0.9418610632596535


#### Linear Regression Model

In [13]:
linreg = LinearRegression() # create a Linear Regression object 

linreg.fit(xtrain, ytrain)

# Predict Response corresponding to Predictors
ytrain_pred = linreg.predict(xtrain)
ytest_pred = linreg.predict(xtest)

# Check the Goodness of Fit (on Train Data)
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", linreg.score(xtrain, ytrain))
print("Mean Squared Error (MSE) \t:", mean_squared_error(ytrain, ytrain_pred))
print()

# Check the Goodness of Fit (on Test Data)
print("Goodness of Fit of Model \tTest Dataset")
print("Explained Variance (R^2) \t:", linreg.score(xtest, ytest))
print("Mean Squared Error (MSE) \t:", mean_squared_error(ytest, ytest_pred))
print()

Goodness of Fit of Model 	Train Dataset
Explained Variance (R^2) 	: 0.9609121380799182
Mean Squared Error (MSE) 	: 947883468.7271221

Goodness of Fit of Model 	Test Dataset
Explained Variance (R^2) 	: -2.099540867215019e+17
Mean Squared Error (MSE) 	: 4.627497432739637e+27



### The best of the Machine Learning Algorithms
#### We will attempt to perform a Gridsearch with inner cross validation in order to find the best parameters for the best estimation model. 

In [28]:
start = tm.time()
param_grid = {"n_estimators": [5, 10], "max_depth": [5]}
gridobj = GridSearchCV(RandomForestRegressor(n_jobs = -1, random_state = 8), param_grid, cv = 5)
gridobj.fit(xtrain, ytrain)
print("Best parameters: {}".format(gridobj.best_params_))
print("Best cross-validation score: {:.2f}".format(gridobj.best_score_))
print("Training set Accuracy : ", gridobj.score(xtrain, ytrain))
print("Test set Accuracy : ", gridobj.score(xtest, ytest))
end = tm.time()
print("\nRunning Time : ", round(end-start, 2))

Best parameters: {'max_depth': 5, 'n_estimators': 10}
Best cross-validation score: 0.66
Training set Accuracy :  0.6510468178869009
Test set Accuracy :  0.6363964463435782

Running Time :  44.33
